# K-Means



O algoritmo K-Means procura un número predeterminado de *clusters* (K) nun dataset multidimensional sen etiquetas. Cada observación debe pertencer a un *cluster*, e está fundamentado en:

*   Cada *cluster* ten un centro (*centroide*) que resulta da media aritmética de todos os puntos do *cluster*.
*   Cada punto do *cluster* debe estar máis cerca do seu centroide que a centroides dos outros *clusters*.

Estas asuncións constitúen a base do modelo K-Means.

## Dependencias

In [ ]:
!pip install matplotlib scikit-learn numpy

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## Dataset

Partimos dun dataset sintético de dúas dimensións. Usamos `make_blobs()` para xerar 300 observacións aleatorias en torno a catro centros e cunha dispersión baixa (`cluster_std`).


In [ ]:
from sklearn.datasets import make_blobs

X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)

In [ ]:
X.shape

Mediante `matplotlib` visualizamos os nosos datos producindo un *scatter plot*.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=10)
plt.xlabel("Eixo X")
plt.ylabel("Eixo Y")
plt.title("Dataset sintético de mostra")
plt.show()

* `s`: determina o tamaño dos puntos na gráfica

https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html

## Escolla de *k*

### Método do cóbado (*elbow method*)

Para estimar un valor optimo para *k* podemos usar o método do cóbado, que consiste en calcular a SSE (*Sum of Squared Errors*) para diferentes valores de *k*. O obxectivo é encontrar o punto onde a diminución do erro comeza a ser menos pronunciada, formando un ángulo semellante a un cóbado. Este punto marca o equilibrio entre complexidade e precisión do modelo, indicando un valor axeitado para *k*.

En `scikit-learn`, o cálculo de SSE gárdase no atributo `inertia_` do obxecto `KMeans`, despois de axustar o modelo.

In [ ]:
from sklearn.cluster import KMeans

sse = []

for k in range(2, 11):
  kmeans2 = KMeans(n_clusters=k, random_state=0)
  kmeans2.fit(X)
  sse.append(kmeans2.inertia_)

Unha vez calculados os valores de SSE para *k* entre 1 e 10, podemos visualizalo nun gráfico para ver en que punto a diminución do erro se estabiliza.

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(2, 11), sse, marker='o')
plt.title('Método do cóbado')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inertia ou SSE')
plt.show()


Outra alternativa é usar o paquete `kneed`de Python.

https://kneed.readthedocs.io/en/stable/




In [ ]:
!pip install kneed

In [ ]:
from kneed import KneeLocator

kl = KneeLocator(range(2, 11), sse, curve="convex", direction="decreasing")

In [ ]:
kl.elbow

### *Silhouette Score*

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_coefficients = []

for k in range(2, 11):
     kmeans3 = KMeans(n_clusters=k)
     kmeans3.fit(X)
     score = silhouette_score(X, kmeans3.labels_)
     silhouette_coefficients.append(score)



In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(2, 11), silhouette_coefficients, marker='o')
plt.title('Silhouette Score')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Silhouette')
plt.show()


## Modelo

Na gráfica puidemos observar que os datos parecen estar distribuídos en catro grupos, e os métodos empregados para estimar *k* confirman esta observación, así que lanzamos K-Means con *k* = 4.

In [ ]:
# Crea un modelo de K-Means con 4 clusters
kmeans = KMeans(n_clusters=4)

# Axustamos o modelo
kmeans.fit(X)

# Produce a saída etiquetada
y_kmeans = kmeans.predict(X)

# Con fit_predict() podemos facer todo xunto
# y_kmeans = kmeans.fit_predict(X)
#
# Podemos acceder ás etiquetas sen chamar a predict(), mediante kmeans.labels_
# kmeans.labels_

In [ ]:
for i in range(5):
  print(f'Punto {i}: {X[i]} no cluster {y_kmeans[i]}')

np.unique(y_kmeans, return_counts=True)

## Resultados

Volvemos xerar unha gráfica de dispersión, agora xa con cada *cluster* nunha cor diferente e marcando tamén os centroides.

* `c`: secuencia de números coas etiquetas dos *clusters*, que son mapeados a cores usando cmap
* `s`: determina o tamaño dos puntos na gráfica
* `cmap`: mapa de cores
* `marker`: marcador usado para pintar as observacións

https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=10, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, marker='x')

In [ ]:
centers

## Número *k* non óptimo

Un dos desafíos coñecidos de K-Means é a necesidade de especificar previamente o número de clusters que queremos obter, xa que o algoritmo non pode determinalo automaticamente a partir dos datos. Se escollemos un valor que non se axusta á estrutura real dos datos, os resultados poden ser pouco representativos ou mesmo enganosos.

Por exemplo, se os datos teñen unha estrutura natural de 4 *clusters*, pero escollemos *k* = 7, o algoritmo dividirá artificialmente os grupos existentes, producindo *clusters* menos interpretables.

In [ ]:
labels = KMeans(7, random_state=0).fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap='managua')